In [1]:
import duckdb
import time
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

In [2]:
con = duckdb.connect(database="tpch_sf1.db")

In [3]:
#2D
query_template = """select s_name, s_address
from supplier,nation
where
	s_suppkey in (
		select ps_suppkey
		from partsupp
		where
			ps_partkey in (
				select p_partkey
				from part
				where p_name like 'navajo%'
			)
			and ps_availqty > (
				select 0.5 * sum(l_quantity)
				from lineitem
				where
					l_partkey = ps_partkey
					and l_suppkey = ps_suppkey
					and l_shipdate >= $shipdate1
					and l_shipdate < $shipdate2
			)
	)
	and s_nationkey = n_nationkey and n_name = 'ETHIOPIA'
order by s_name;
"""

In [4]:
min_shipdate = con.sql("SELECT MIN(l_shipdate) FROM lineitem").fetchone()[0]
max_shipdate = con.sql("SELECT MAX(l_shipdate) FROM lineitem").fetchone()[0]
(min_shipdate, max_shipdate)

(datetime.date(1992, 1, 2), datetime.date(1998, 12, 1))

In [5]:
table = []
NUM_TRIALS = 3

for shipdate1 in tqdm(list(pd.date_range(min_shipdate, max_shipdate, freq='30D'))):
    for shipdate2 in tqdm(list(pd.date_range(min_shipdate, max_shipdate, freq='30D')), leave=False):
        params = {'shipdate1': shipdate1, 'shipdate2': shipdate2}
        row = dict()
        for trial in range(NUM_TRIALS):
            start = time.time()
            res = con.sql(query_template, params=params)
            elapsed = time.time() - start
            row[f'elapsed_{trial}'] = elapsed
            # If we want to, save the result for a sanity check
            # row[f'res_{trial}'] = res.fetchall()
        table.append({**params, **row})

table = pd.DataFrame(table)
table

  0%|          | 0/85 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

...

  0%|          | 0/85 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

RuntimeError: Query interrupted

In [ ]:
con.close()